In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time

Main category links extraction

In [2]:
path = "C:\Program Files (x86)\chromedriver.exe"
#url = "https://www.massimodutti.com/tr"
massimo_links = ["https://www.massimodutti.com/tr/","https://www.massimodutti.com/tr/erkek-n1343"]
cat_links = [] ##store category links to loop over it afterwards
for url in massimo_links:
    driver = webdriver.Chrome(path)
    driver.get(url)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    soup = BeautifulSoup(driver.page_source, "html.parser")
    for product in soup.find_all(class_="link-line has-ellipsis"):
        cat_links.append(product.get("href"))
driver.close()
    

deduplication, check afterwards

In [3]:
remove_list = ["/tr/women/total-look-n2271","/tr/c1887021.html","/tr/women/back-to-work-n2374","/tr/c2070730.html","/tr/men/back-to-work-n2376"]

cat_links = list(set(cat_links) - set(remove_list))

Scraping

In [21]:
path = "C:\Program Files (x86)\chromedriver.exe"
prices = []
for cat in cat_links: 
    url = f'https://www.massimodutti.com{cat}'
    driver = webdriver.Chrome(path)
    driver.get(url)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 0.8 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    soup = BeautifulSoup(driver.page_source, "html.parser")
    for x in soup.find_all(class_ = "card9-wrap-details"):
        if x.find('a', {'class':'pointer'}) is not None:
            prod_name = x.find('a', {'class':'pointer'}).text
            price = x.find(class_ = "is-flex flex-wrap").text.split()[0]
            try:
                disc_price = x.find(class_ = "js-future-price c-sale").text.split()[0]
            except:
                disc_price = "-"            
            massimo_dict = {
            'category':cat,
            'name':prod_name,
            'price':price,
            'disc_price':disc_price
            }
            prices.append(massimo_dict)
driver.close()
massimo_prods = pd.DataFrame(prices)
massimo_prods

,category,name,price,disc_price
0,/tr/erkek/homewear-n1531,Uzun pijama altlı puantiyeli kısa kollu pijam...,"899,00",-
1,/tr/erkek/homewear-n1531,Pamuklu klasik uzun pijama takımı,"999,00",-
2,/tr/erkek/homewear-n1531,Uzun pijama altlı puantiyeli kısa kollu pijam...,"899,00",-
3,/tr/erkek/homewear-n1531,Uzun pijama altlı çizgili uzun kollu pijama t...,"899,00",-
4,/tr/erkek/homewear-n1531,Çizgili pijama altı ve kısa kollu t-shirt,"899,00",-
...,...,...,...,...
296,/tr/men/loungewear-collection-n2328,Dikiş detaylı bisiklet yaka sweatshirt,"899,00",-
297,/tr/men/loungewear-collection-n2328,Dokulu %100 pamuklu polo t-shirt,"529,00",-
298,/tr/men/loungewear-collection-n2328,Dokulu %100 pamuklu polo t-shirt,"529,00",-
299,/tr/men/loungewear-collection-n2328,Dokulu %100 pamuklu polo t-shirt,"529,00",-


Format change

In [22]:
massimo_prods["has_discount"]= massimo_prods.apply(lambda x: x["disc_price"] != "-", axis=1) ##add discount availability feature
massimo_prods["disc_price"]= massimo_prods.apply(lambda x: x['price'] if x["disc_price"] == "-" else x['disc_price'], axis=1) ##disc_price "-" structure change for float conversion
massimo_prods["price"] = massimo_prods["price"].apply(lambda x: x.replace(",00","")) # " TL" deletion
massimo_prods["disc_price"] = massimo_prods["disc_price"].apply(lambda x: x.replace(",00",""))# " TL" deletion
massimo_prods["price"] = massimo_prods["price"].apply(lambda x: x.replace(".","")) # "." conversion
massimo_prods["disc_price"] = massimo_prods["disc_price"].apply(lambda x: x.replace(".",""))# "." conversion
massimo_prods["price"] = massimo_prods.price.astype(float)
massimo_prods["disc_price"] = massimo_prods.disc_price.astype(float)
massimo_prods


,category,name,price,disc_price,has_discount
0,/tr/erkek/homewear-n1531,Uzun pijama altlı puantiyeli kısa kollu pijam...,899.0,899.0,False
1,/tr/erkek/homewear-n1531,Pamuklu klasik uzun pijama takımı,999.0,999.0,False
2,/tr/erkek/homewear-n1531,Uzun pijama altlı puantiyeli kısa kollu pijam...,899.0,899.0,False
3,/tr/erkek/homewear-n1531,Uzun pijama altlı çizgili uzun kollu pijama t...,899.0,899.0,False
4,/tr/erkek/homewear-n1531,Çizgili pijama altı ve kısa kollu t-shirt,899.0,899.0,False
...,...,...,...,...,...
296,/tr/men/loungewear-collection-n2328,Dikiş detaylı bisiklet yaka sweatshirt,899.0,899.0,False
297,/tr/men/loungewear-collection-n2328,Dokulu %100 pamuklu polo t-shirt,529.0,529.0,False
298,/tr/men/loungewear-collection-n2328,Dokulu %100 pamuklu polo t-shirt,529.0,529.0,False
299,/tr/men/loungewear-collection-n2328,Dokulu %100 pamuklu polo t-shirt,529.0,529.0,False
